# Manipulation Tactic Detector - Training Notebook

This notebook fine-tunes the `j-hartmann/emotion-english-distilroberta-base` model on the ManTacAi dataset to detect manipulation tactics.

## Instructions
1. Upload the `dataset_augmented/v5_training_data_final.json` file to your Kaggle environment as a dataset named `v5-training-data-final`.
2. Run all cells to train the model.
3. Download the `manipulation_tactic_detector_model.zip` file at the end.

In [ ]:
# Install dependencies.
# NOTE: You will likely see "ERROR: pip's dependency resolver..." regarding libraries like 
# 'cudf', 'bigframes', 'pyarrow', etc. 
# PLEASE IGNORE THESE ERRORS. They are for libraries we are NOT using.
# As long as the check below passes, you are good to go.
!pip install -q -U transformers datasets evaluate accelerate scikit-learn

print("\n--- Verifying Installation ---")
try:
    import transformers
    import datasets
    import evaluate
    import accelerate
    import sklearn
    print("✅ Success! All core dependencies imported correctly.")
    print("You can safely proceed to the next cells.")
except ImportError as e:
    print(f"❌ Critical Error: {e}")
    raise e

In [ ]:
import os
# Suppress tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import json
import numpy as np
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, f1_score

## 1. Configuration & Data Loading

In [ ]:
# Label Mapping (Must match src/utils/config.py)
id2label = {
    0: "ethical_persuasion",
    1: "gaslighting", 
    2: "guilt_tripping",
    3: "deflection",
    4: "stonewalling",
    5: "belittling_ridicule",
    6: "love_bombing",
    7: "threatening_intimidation", 
    8: "passive_aggression",
    9: "appeal_to_emotion",
    10: "whataboutism",
    11: "neutral_conversation",
    12: "coercive_control"
}
label2id = {v: k for k, v in id2label.items()}

# Load Dataset
# Updated path for Kaggle input directory
dataset_path = "/kaggle/input/v5-training-data-final/v5_training_data_final.json"

try:
    with open(dataset_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Successfully loaded dataset. Keys: {list(data.keys())}")
except FileNotFoundError:
    print(f"ERROR: File '{dataset_path}' not found. Please ensure the dataset is added to the notebook.")
    # Fallback for local testing if needed, or stop
    raise

def create_dataset_object(data_list):
    # Filter out any items with unknown labels
    valid_items = [
        item for item in data_list 
        if item["manipulation_tactic"] in label2id
    ]
    if len(valid_items) < len(data_list):
        print(f"Warning: Filtered out {len(data_list) - len(valid_items)} items with unknown labels.")
        
    return Dataset.from_list([
        {
            "text": item["text"],
            "label": label2id[item["manipulation_tactic"]]
        } 
        for item in valid_items
    ])

# Handle different key names for validation split
val_key = "validation" if "validation" in data else "val"

if val_key in data and "test" in data:
    raw_datasets = DatasetDict({
        "train": create_dataset_object(data["train"]),
        "validation": create_dataset_object(data[val_key]),
        "test": create_dataset_object(data["test"])
    })
else:
    print("Splitting 'train' set as validation/test sets were not found.")
    full_ds = create_dataset_object(data["train"])
    splits = full_ds.train_test_split(test_size=0.2, seed=42)
    test_val = splits["test"].train_test_split(test_size=0.5, seed=42)
    raw_datasets = DatasetDict({
        "train": splits["train"],
        "validation": test_val["train"],
        "test": test_val["test"]
    })

print(raw_datasets)

## 2. Preprocessing

In [ ]:
model_checkpoint = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 3. Model Setup

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # Essential for replacing the head
)
# Note: You will see a warning about weights not being initialized. 
# This is EXPECTED because we are replacing the emotion classification head with a new one.

## 4. Training

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

## 5. Evaluation & Saving

In [ ]:
eval_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test Results: {eval_results}")

# Save Model to Kaggle working directory
save_path = "/kaggle/working/manipulation_tactic_detector_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

# Zip for download
!zip -r manipulation_tactic_detector_model.zip /kaggle/working/manipulation_tactic_detector_model